# 步骤一：（仅carol）前置部署

在体验TrustedFlow之前，carol需要先部署CapsuleManager。CapsuleManager是TrustedFlow的核心模块之一，负责授权管理和密钥管理，CapsuleManager被设计为运行在TEE上，机构可以对CapsuleManager的代码和环境进行远程认证。

## 选项一：仿真模式部署CapsuleManager

如果你希望使用仿真模式进行体验，则可以按下列说明进行。

### 1. 运行CapsuleManager镜像

```bash
docker run -it --name capsule-manager-sim --network=host secretflow/capsule-manager-sim-ubuntu22.04:latest bash
```

### 2. 启动CapsuleManager

CapsuleManager 默认会启用mTLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)：
```bash
./capsule_manager --server-cert-key-path <SERVER_CERT_KEY_PATH> \
                  --server-cert-path <SERVER_CERT_PATH>         \
                  --client-ca-cert-path <CLIENT_CA_CERT_PATH>
```
如果不希望开启mTLS，可以添加启动参数`--enable-tls false` 来**关闭mTLS功能**（注意关闭mTLS是不安全的，生产环境建议启动mTLS）：
```bash
./capsule_manager --enable-tls false
```
默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号。

## 选项二：在SGX机器上运行CapsuleManager

如果您希望在SGX机器上运行，则可以按下列说明进行。

### 1. 检查sgx环境
**宿主机**上执行以下命令确保存在/dev/sgx_enclave和/dev/sgx_provision。
```bash
ls /dev | grep sgx
```

### 2. 运行镜像

```bash
docker run -it --name capsule-manager-sgx --network=host -v /dev/sgx_enclave:/dev/sgx/enclave -v /dev/sgx_provision:/dev/sgx/provision --privileged=true secretflow/capsule-manager-sgx-ubuntu22.04:latest bash
```

### 3. 修改 PCCS 配置

> 提示：如果您还没有PCCS服务，则可以参考[部署PCCS](../architecture/tee/sgx.md#如何部署pccs服务)。


1. 修改PCCS的配置文件/etc/sgx_default_qcnl.conf，把`pccs_url`配置为PCCS的实际部署服务地址。如果您的PCCS服务没有开启tls，把`use_secure_cert`设置为false。

```bash
# PCCS server address
"pccs_url": "https://localhost:8081/sgx/certification/v4/",

# To accept insecure HTTPS certificate, set this option to FALSE
"use_secure_cert": false

```

2. 把/etc/sgx_default_qcnl.conf复制到occlum的image中

```bash
cd /home/admin/occlum_instance
cp /etc/sgx_default_qcnl.conf ./image/etc/sgx_default_qcnl.conf
```


### 4. 生成私钥后，使用私钥进行build。

您首先需要生成私钥，然后使用以下命令构建occlum。生成私钥可以参考下列脚本，生成的私钥保存在当前目录的private_key.pem。请妥善保存您的私钥，不要泄露给其他人。

```bash
openssl genrsa -3 -out private_key.pem 3072
```

生成公私钥后，使用私钥构建occlum。

```bash
occlum build -f --sign-key private_key.pem
```

### 5. 运行服务

CapsuleManager 默认会启用mTLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)


> 默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号

```bash
occlum run /bin/capsule_manager --server-cert-key-path <SERVER_CERT_KEY_PATH> \
                                --server-cert-path <SERVER_CERT_PATH>         \
                                --client-ca-cert-path <CLIENT_CA_CERT_PATH>
```

如果不希望开启mTLS，可以添加启动参数`--enable-tls false` 来**关闭mTLS功能**（注意关闭mTLS是不安全的，生产环境建议启动mTLS）：
```bash
occlum run /bin/capsule_manager --enable-tls false
```

### 7. 获取CapsuleManager的mrenclave

执行下列命令可以获得CapsuleManager的mrenclave，mrenclave是表征CapsuleManager代码、数据、运行环境等的度量值，详细解释见[Enclave](../architecture/tee/sgx.md#enclave)。

输出内容为一串小写的十六进制字符串，您可以保存下来，后续步骤会使用到。

```bash
occlum print mrenclave
```

## 选项三：TDX模式部署CapsuleManager

如果你希望在TDX的TD中进行体验，则可以按下列说明进行。

### 1. 检查环境
**TD VM**中执行以下命令确保存在/dev/tdx_guest。**请注意，如果您的环境中存在的是/dev/tdx-guest而非/dev/tdx_guest，则说明您的tdx版本较老，需要更新。**
```bash
ls /dev | grep tdx
```

### 2. 运行CapsuleManager镜像
在**TD VM**中运行CapsuleManager镜像。

```bash
docker run -it --name capsule-manager-tdx --network=host -v /dev/tdx_guest:/dev/tdx_guest --privileged=true secretflow/capsule-manager-tdx-ubuntu22.04:latest bash
```

### 3. 修改 PCCS 配置

> 提示：如果您还没有PCCS服务，则可以参考[部署PCCS](../architecture/tee/sgx.md#如何部署pccs服务)。


修改PCCS的配置文件/etc/sgx_default_qcnl.conf，把`pccs_url`配置为PCCS的实际部署服务地址。如果您的PCCS服务没有开启tls，把`use_secure_cert`设置为false。

```bash
# PCCS server address
"pccs_url": "https://localhost:8081/sgx/certification/v4/",

# To accept insecure HTTPS certificate, set this option to FALSE
"use_secure_cert": false

```

### 4. 启动CapsuleManager

CapsuleManager 默认会启用mTLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)：
```bash
./capsule_manager --server-cert-key-path <SERVER_CERT_KEY_PATH> \
                  --server-cert-path <SERVER_CERT_PATH>         \
                  --client-ca-cert-path <CLIENT_CA_CERT_PATH>
```
如果不希望开启mTLS，可以添加启动参数`--enable-tls false` 来**关闭mTLS功能**（注意关闭mTLS是不安全的，生产环境建议启动mTLS）：
```bash
./capsule_manager --enable-tls false
```
默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号。

### 5. 获取CapsuleManager所在的VM度量值

目前暂无简易工具可以获取度量值，您可以在后续步骤中获取远程认证报告并记录其中携带的度量值用于后验。

## 选项四：CSV模式部署CapsuleManager

如果你希望在海光CSV虚拟机中进行体验，则可以按下列说明进行。

### 1. 检查环境
**CSV VM**中执行以下命令确保存在/dev/csv-guest。
```bash
ls /dev/ | grep csv
```

假如您的环境中不存在/dev/csv-guest，可以下载海光开源的[csv-guest驱动源码](https://gitee.com/anolis/hygon-devkit/blob/master/csv/attestation/csv-guest.c)并编译和安装该驱动。
这里给出Makefile的示例:
```makefile
obj-m += csv-guest.o

all:
        make -C /lib/modules/$(shell uname -r)/build M=$(PWD) modules

clean:
        make -C /lib/modules/$(shell uname -r)/build M=$(PWD) clean
```

您可以将下载的源码和以上Makefile文件放在同一个文件夹下并执行:
```bash
make
sudo insmod csv-guest.ko
```

### 2. 运行CapsuleManager镜像
在**CSV VM**中运行CapsuleManager镜像。

```bash
docker run -it --name capsule-manager-csv --network=host -v /dev/csv-guest:/dev/csv-guest --privileged=true secretflow/capsule-manager-csv-ubuntu22.04:latest bash
```

### 3. 启动CapsuleManager

CapsuleManager 默认会启用mTLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)：
```bash
./capsule_manager --server-cert-key-path <SERVER_CERT_KEY_PATH> \
                  --server-cert-path <SERVER_CERT_PATH>         \
                  --client-ca-cert-path <CLIENT_CA_CERT_PATH>
```
如果不希望开启mTLS，可以添加启动参数`--enable-tls false` 来**关闭mTLS功能**（注意关闭mTLS是不安全的，生产环境建议启动mTLS）：
```bash
./capsule_manager --enable-tls false
```
默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号。

### 4. 获取CapsuleManager所在的VM度量值

目前暂无简易工具可以获取度量值，您可以在后续步骤中获取远程认证报告并记录其中携带的度量值用于后验。